# Exercise 1.3 - the guvectorize decorator

## Objectives

- See how to specify input and output arrays and scalars to guvectorized functions
- Observe how guvectorized functions are applied to arrays of different shapes

## Layout Specifications

Define the function `g` as follows:

In [ ]:
# Very simple example:
from numba import guvectorize, int64, float64

@guvectorize([(int64[:], int64[:], int64[:])], '(n),()->(n)')
def g(x, y, res):
    for i in range(x.shape[0]):
        res[i] = x[i] + y[0]

## Note

The layout specification suggests that `x` is an n-length vector and that `y` is a scalar. However, the signature for `y` is `int64[:]` as if it is an array. This is because scalars must be passed like arrays. To access their values, the first element of this “array” must be dereferenced, as in `y[0]` in the example above.

## Call examples

Try calling `g` with different arguments to see how Numpy dispatches the function over different arguments depending on their shapes:

In [ ]:
import numpy as np

a = np.arange(6)
a

In [ ]:
g(a, 10)

In [ ]:
b = np.arange(6).reshape(2, 3)
b

In [ ]:
g(b, 10)

In [ ]:
g(b, np.array([10, 20]))

## A more complicated example

Having visibility of an entire sub-array allows complex operations to be performed. For example, to compute the moving mean of sub-arrays:

In [ ]:
# Moving mean example
@guvectorize([(float64[:], int64[:], float64[:])], '(n),()->(n)')
def move_mean(a, window_arr, out):
    window_width = window_arr[0]
    asum = 0.0
    count = 0
    for i in range(window_width):
        asum += a[i]
        count += 1
        out[i] = asum / count
    for i in range(window_width, len(a)):
        asum += a[i] - a[i - window_width]
        out[i] = asum / count

arr = np.arange(20, dtype=np.float64).reshape(2, 10)
move_mean(arr, 3)

## Further Reading

For additional information on Numpy gufuncs, see the Generalized Universal Function API documentation: http://docs.scipy.org/doc/numpy/reference/c-api.generalized-ufuncs.html

## Exercises (optional)

- Write a gufunc that computes both the minimum and maximum of input arrays.
- Write a gufunc that performs batch matrix multiplication.

